In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import logm, expm, eigh
import os
import gseapy as gp

In [2]:
def is_spd(A, tol=1e-8):
    # Check symmetry
    if not np.allclose(A, A.T, atol=tol):
        return False
    # Check eigenvalues > 0
    eigvals = np.linalg.eigvalsh(A)
    return np.all(eigvals > tol)

def project_to_spd(A, tol=1e-8):
    # Make symmetric
    A = (A + A.T) / 2
    eigvals, eigvecs = eigh(A)
    eigvals_clipped = np.clip(eigvals, tol, None)  # set eigenvalues < tol to tol
    return eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

def make_psd(K, min_eig=1e-6):
    K = (K + K.T) / 2
    eigvals = np.linalg.eigvalsh(K)
    if np.min(eigvals) < min_eig:
        K += np.eye(K.shape[0]) * (min_eig - np.min(eigvals))
    return K

def weighted_log_euclidean_mean(kernels, weights):
    weights = np.array(weights)
    weights = weights / weights.sum()
    
    log_sum = np.zeros_like(kernels[0])
    for w, K in zip(weights, kernels):
        K = make_psd(K)
        log_K = logm(K)
        # Handle potential complex results more carefully
        if np.iscomplexobj(log_K) and np.allclose(log_K.imag, 0, atol=1e-10):
            log_K = log_K.real
        log_sum += w * log_K
    
    return expm(log_sum)

def read_data(disease, dga, features):
    columns_to_keep = [col for col in features.columns if col.startswith('feature') or col.startswith('string')]
    df = features[columns_to_keep]

    pos_genes_list = dga[dga['disease_id']==disease]['string_id']
    df['label'] = df['string_id'].isin(pos_genes_list).astype(int)

    # X = df.loc[:, df.columns.str.startswith("feature_")].to_numpy()
    y = df['label'].to_numpy()
    df.set_index('string_id', inplace=True)
    df.drop(columns='label', inplace=True)
    return df, y

def read_data_timecut(disease, dga, features,time):
    pos_genes_list = dga[dga['disease_id']==disease]['string_id']
    columns_to_keep = [col for col in features.columns if 'feature' in col or col.startswith('string')]
    df = features[columns_to_keep]
    df['label'] = df['string_id'].isin(pos_genes_list).astype(int)
    df['test'] = df['string_id'].isin(dga[(dga['disease_id'] == disease) & (dga['first_pub_year'] > time)]['string_id']).astype(int)

    # X = df.loc[:, df.columns.str.startswith("feature_")].to_numpy()
    y = df['label'].to_numpy()
    df.set_index('string_id', inplace=True)
    df.drop(columns='label', inplace=True)
    return df, y


def get_feature(root, feature_name):
    if feature_name == 'ppi_align':
        feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_emb_aligned.csv'))
    elif feature_name == 'ppi':
        feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_emb.csv'))
    elif feature_name == 'biograd':
        feature_df = pd.read_csv(os.path.join(root,'data/biograd/biograd_full_emb.csv'))
    elif feature_name == 'prose':
        feature_df = pd.read_csv(os.path.join(root, 'data/prose/data/prose_emb_full.csv'))
    # elif feature_name == 'ppi_400':
    #     feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_400_emb.csv'))
    elif feature_name == 'ppi_2016':
        feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_2016_emb.csv'))
    elif feature_name == 'ppi_2019':
        feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_2019_emb.csv'))
    # elif feature_name == 'ppi_2013':
    #     feature_df = pd.read_csv(os.path.join(root,'data/ppi_full_2013_emb.csv'))
    elif feature_name == 'uniport':
        feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/seq_emb/human_uniport_seqemb.csv'))
    # elif feature_name == 't5_align':
    #     feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/T5/T5_align.csv'))
    elif feature_name == 'gene2vec':
        feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/expression_emb/exp_emb.csv'))
        # feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/GENE2VEC/GENE2VEC_align.csv'))
    
    elif feature_name == 'scgpt':
        feature_df = pd.read_csv(os.path.join(root,'data/scgpt/scgpt_full.csv'))
        # feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/expression_emb/SCGPT-HUMAN/scgpt.csv'))
    elif feature_name == 'bioconcept':
        # feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/text_minning_2015/text_features.csv'))
        # feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/txt/BIOCONCEPTVEC-FASTTEXT/text.csv'))
        feature_df = pd.read_csv(os.path.join(root,'data/bioconcept/bioconcept_full.csv'))
    # elif feature_name == 'go_svd':
    #     feature_df = pd.read_csv(os.path.join(root,'data/GO/GO_2023_features.csv'))
    # elif feature_name == 'go_all':
    #     feature_df = pd.read_csv(os.path.join(root,'data/GO/GO_2023_all_features_aligned.csv'))
    elif feature_name == 'esm2':
        feature_df = pd.read_csv(os.path.join(root,'data/esmfold/esm2.csv'))
        # feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/ESM2/ESM2_align.csv'))
    # elif feature_name == 'MASHUP':
    #     feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/MASHUP/MASHUP_align.csv'))
    # elif feature_name == 'GENEPT_MODEL3':
    #     feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/GENEPT_MODEL3/GENEPT_MODEL3_align.csv'))    
    # elif feature_name == 'GF_12L95M':
    #     feature_df = pd.read_csv(os.path.join(root,'data/pre_processed_features/GF_12L95M/GF_12L95M_align.csv')) 

    return feature_df

import pickle
with open('/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb/2023/name_convert.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
stringId2name,name2stringId,aliases2stringId = loaded_data
del name2stringId,aliases2stringId

def enriched_set(input_stringids,time):
    gene_names = [stringId2name.get(sid) for sid in input_stringids if stringId2name.get(sid) is not None]
    if time == 2019:
        enrich_db = ['GO_Biological_Process_2021','GO_Cellular_Component_2021','GO_Molecular_Function_2021','KEGG_2019_Human']
    elif time == 2017:
        enrich_db = ['GO_Biological_Process_2021','GO_Cellular_Component_2021','GO_Molecular_Function_2021','KEGG_2016']
    enr = gp.enrichr(gene_list=gene_names,
                    gene_sets=enrich_db,
                    organism='human', 
                    outdir=None, 
                    )
    enr_df = enr.results
    result_terms = enr_df.loc[enr_df['Adjusted P-value'] < 0.01, ['Gene_set', 'Term']]
    return set(map(tuple, result_terms.values))

def calculate_jac_sim(enrich_1, enrich_2):
    intersection = enrich_1 & enrich_2
    union = enrich_1 | enrich_2
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity

In [3]:
time_spilt = True
time = 2019
feature_list = ['ppi_2019','bioconcept']
root = '/itf-fi-ml/shared/users/ziyuzh/svm'

merged_df = None
for feature in feature_list:
    feature_df = get_feature(root, feature)
    # Rename columns starting with 'feature'
    feature_df.rename(columns={
        col: f"{feature}_{col}" if col.startswith('feature') else col
        for col in feature_df.columns
    }, inplace=True)
    # Merge iteratively to avoid keeping all DataFrames
    if merged_df is None:
        merged_df = feature_df
    else:
        merged_df = pd.merge(merged_df, feature_df, on='string_id', how='inner')
    del feature_df  # Free memory

all_df = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/disgent_2020/timecut/disgent_with_time.csv')
all_df = all_df[all_df['string_id'].isin(merged_df['string_id'])]

if time_spilt:
    selected_diseases = []
    for disease_id in all_df['disease_id'].unique():
        sub_df = all_df[all_df['disease_id']==disease_id]
        if len(sub_df) < 15:
            continue
        else:
            # print(type(time),type(sub_df['first_pub_year'].max()))
            if sub_df['first_pub_year'].max() > time and sub_df['first_pub_year'].min() <= time and len(sub_df[sub_df['first_pub_year']<time]) >=5:
                selected_diseases.append(disease_id)
else:
    selected_diseases = (
        all_df.groupby('disease_id')
        .filter(lambda x: (len(x) > 15))
        ['disease_id']
        .unique()
        .tolist())
print(feature_list, len(selected_diseases),len(merged_df))
all_results = []

for disease in selected_diseases[:1]:
    print(disease,len(all_df[all_df['disease_id']==disease]))
    if time_spilt:
        df, y = read_data_timecut(disease, all_df, merged_df,time)
    else:
        df, y = read_data(disease, all_df, merged_df,time)

['ppi_2019', 'bioconcept'] 42 12343
ICD10_C16 69


In [4]:
test_idx = df[df['test']==1].index
train_idx = df[y==1].index.difference(test_idx)
df.drop(columns='test', inplace=True)

train_pos_df = df.loc[train_idx]
test_pos_df = df.loc[test_idx]
neg_num = 5*len(train_pos_df)

if 'linear_fused' in feature_list:
    feature_list.remove('linear_fused')
if 'geo_fused' in feature_list:
    feature_list.remove('geo_fused')

In [5]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import NearestNeighbors
from sklearn import svm

In [6]:
if 'linear_fused' in feature_list:
    feature_list.remove('linear_fused')
if 'geo_fused' in feature_list:
    feature_list.remove('geo_fused')


# Work with DataFrames to maintain indices
neg_df = df[y == 0]

# Randomly select 'neg_num' samples from negative class
train_neg_df = neg_df.sample(n=neg_num, random_state=42)

# Get the all negative samples
test_neg_df = neg_df

# Combine positive and negative samples for training
train_df = pd.concat([train_pos_df, train_neg_df])
test_df = pd.concat([test_pos_df, test_neg_df])

X_train_mats = []
X_test_mats = []
for feature_name in feature_list:
    select_columns = [col for col in df.columns if col.startswith(feature_name)]
    X_train_mats.append(train_df[select_columns].values)
    X_test_mats.append(test_df[select_columns].values)

feature_weights = [1 / len(feature_list)] * len(feature_list)

y_train = np.array([1] * len(train_pos_df) + [0] * len(train_neg_df))
y_test = np.array([1] * len(test_pos_df) + [0] * len(test_neg_df))

In [7]:
kernels_all = []
kernels_train = []
kernels_test = []

# For each feature set
for X_tr, X_te in zip(X_train_mats, X_test_mats):
    X_all = np.concatenate([X_tr, X_te], axis=0)

    nbrs = NearestNeighbors(n_neighbors=2).fit(X_all)
    distances, _ = nbrs.kneighbors(X_all)
    avg_nn_dist = np.mean(distances[:, 1])  # skip self-distance
    gamma = 1 / (2 * avg_nn_dist ** 2)
    K_full = rbf_kernel(X_all, X_all, gamma=gamma)
    kernels_all.append(K_full)

    n_train = len(X_tr)
    kernels_train.append(K_full[:n_train, :n_train])
    kernels_test.append(K_full[n_train:, :n_train])

In [8]:
test_indices = test_df.index.values
enrich_train_genes = train_pos_df.index.values
enrich_train_set = enriched_set(enrich_train_genes,time)

pathway_overlap = []
for feature_index, feature_name in enumerate(feature_list):
    best_svm = svm.SVC(kernel='precomputed')
    best_svm.fit(kernels_train[feature_index], y_train)
    y_scores = best_svm.decision_function(kernels_test[feature_index])

    ############################## Add results to df here 
    enrich_test_genes = test_indices[np.argsort(y_scores)[::-1]][:200]
    enrich_feature_test = enriched_set(enrich_test_genes,time)
    jac_sm = calculate_jac_sim(enrich_train_set,enrich_feature_test)
    pathway_overlap.append(jac_sm)

total = sum(pathway_overlap)
feature_weights = [v / total for v in pathway_overlap]
    

In [9]:
K_linear_all = sum(w * K_train_i for w, K_train_i in zip(feature_weights, kernels_all))
kernels_train.append(K_linear_all[:n_train, :n_train])
kernels_test.append(K_linear_all[n_train:, :n_train])
feature_list.append('linear_fused')

# K_geo_all = weighted_log_euclidean_mean(kernels_all, feature_weights)
# kernels_train.append(K_geo_all[:n_train, :n_train])
# kernels_test.append(K_geo_all[n_train:, :n_train])
# feature_list.append('geo_fused')

In [ ]:
for feature_index, feature_name in enumerate(feature_list[-1:]):
    best_svm = svm.SVC(kernel='precomputed')
    best_svm.fit(kernels_train[feature_index], y_train)
    y_scores = best_svm.decision_function(kernels_test[feature_index])
    ############################ add resluts to df here

In [12]:
pathway_overlap

[0.09916589434661724, 0.26582278481012656]

In [13]:
feature_list, feature_weights

(['ppi_2019', 'bioconcept', 'linear_fused'],
 [0.27169580869118026, 0.7283041913088196])